i made class named Sensor have the necessary arguments and informations
each type of sensors will inherit from the Sensor class with his own arguments

we have IOT class and it's our queue class  and have as arguements :cores /max capacity
in our constructor function we intilize the necessary attributes 
than we call the env.process with self.sensor_arrival(sensor_type) 5 times each time have different sensor type name


2-senors_arrival :
in this function we check type of sensor and make object from one of the 5 sensors classes (have the same name as argument : sensortype)
after we make sensor we call process_sensors function

3-sensors_process : 
in this function we wait until we have Resource(one of our cores) available than if we have sensors waiting in queue we choose one of them to be processed by the Resource
we choose the sensor by his Priority than time  (if two sensors have the same priority do FIFO)

4-stats:
when we were doing the simulation (arrivals -process - waiting,size,capacity in arrival ..etc )
every time we give the object the specific information than we just put all the sensors in dataframe and we show as columns each sensor information

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import simpy 
import scipy.stats as st
TIME_STEPS = 24*3600

class Infos():
    def __init__(self,arrival_time,waiting,capacity,test) -> None:
        self.waiting = waiting
        self.capacity = capacity
        self.arrival_time = arrival_time
        self.wait = None
        self.done = None
        self.core = None
        self.test = test
        self.successed = "rejected"


class Sensor():
    id = 0
    def __init__(self,sen_type,priority,size,processing_time,arrival_rate):
        Sensor.id +=1
        self.id = Sensor.id
        self.sen_type = sen_type
        self.size = size
        self.processing_time = processing_time
        self.priority = priority
        self.arrival_rate = arrival_rate
        self.infos :Infos = None



class IntrusionDetector(Sensor):
    def __init__(self):
        size = st.norm(128,32).rvs()
        process_t = st.expon(0,15).rvs()
        arrival_rate = 30
        super().__init__("IntrusionDetector",1,size=size,processing_time=process_t,arrival_rate=arrival_rate)

class GasCarbonDetector(Sensor):
    def __init__(self):
        size = 1
        arrival_rate = 60
        process_t = st.expon(0,5).rvs()
        super().__init__("GasCarbonDetector",1,size=size,processing_time=process_t,arrival_rate=arrival_rate)

class VocalDetector(Sensor):
    def __init__(self):
        size = 256
        process_t = st.expon(0,10).rvs()
        arrival_rate = st.expon(0,3600/12).rvs()
        super().__init__("VocalDetector",2,size=size,processing_time=process_t,arrival_rate=arrival_rate)

class Camera(Sensor):
    def __init__(self):
        size = st.norm(2000,512).rvs()
        process_t = st.expon(0,15).rvs()
        arrival_rate = 10
        super().__init__("Camera",3,size=size,processing_time=process_t,arrival_rate=arrival_rate)

class Temperature(Sensor):
    def __init__(self):
        size = 1
        process_t = st.expon(0,5).rvs()
        arrival_rate = 180
        super().__init__("Temperature",4,size=size,processing_time=process_t,arrival_rate=arrival_rate)



class IOT():
    def __init__(self,cores=["core1","core2","core3","core4"],max_capacity=128000):
        self.cores = cores.copy()
        self.queue = []
        self.queue_capacity = 0
        self.max_capacity = max_capacity
        self.num_cores = len(self.cores)
        self.sensors = []
        self.env = simpy.Environment()
        print(self.num_cores)
        self.res_cores = simpy.Resource(self.env,self.num_cores)
        self.gen = self.env.process(self.sensors_arrival("IntrusionDetector"))
        self.gen = self.env.process(self.sensors_arrival("GasCarbonDetector"))
        self.gen = self.env.process(self.sensors_arrival("VocalDetector"))
        self.gen = self.env.process(self.sensors_arrival("Camera"))
        self.gen = self.env.process(self.sensors_arrival("Temperature"))
    
    def get_capacity(self):
        s = 0
        
        for sen in self.queue:
            s += sen.size
        return s
    
    def get_sensor(self):
        mini = 0
        for idx,sensor in enumerate(self.queue):
            if sensor.priority < self.queue[mini].priority :
                mini = idx
        return mini
    


    def sensors_arrival(self,sensortype):
        while True :

            if sensortype == "IntrusionDetector" :
                sensor=IntrusionDetector()

            elif sensortype == "GasCarbonDetector" :
                sensor=GasCarbonDetector()

            elif sensortype == "Camera" :
                sensor=Camera()
                
            elif sensortype == "VocalDetector" :
                sensor=VocalDetector()
            
            elif sensortype == "Temperature" :
                sensor = Temperature()
            
            

            arrival_rate = sensor.arrival_rate
            yield self.env.timeout(arrival_rate)
            sensor.infos = Infos(self.env.now,len(self.queue),self.get_capacity(),"zit")
            
            self.sensors.append(sensor)
            if sensor.size < (self.max_capacity - self.get_capacity()) :
                sensor.infos.successed = "successed"
                self.queue.append(sensor)
                
            
            self.env.process(self.process_sensors())



                
    
    def process_sensors(self):
        
        with self.res_cores.request() as req :
            
            yield req
            
            if len(self.queue) > 0 :
            
                sensor = self.queue.pop(self.get_sensor())
                sensor.infos.wait = self.env.now
                core = self.cores.pop(0)
                sensor.infos.core = core
                process_rate = sensor.processing_time
                
                yield self.env.timeout(process_rate)

                sensor.infos.done = self.env.now
                
                self.cores.append(core)
            
            
            

    
    def stats(self):
        data ={
            "Id":[s.id for s in self.sensors],
            "type":[s.sen_type for s in self.sensors],
            "arrival_time":[s.infos.arrival_time for s in self.sensors],
            "t_end_waiting":[s.infos.wait for s in self.sensors],
            "t_out":[s.infos.done for s in self.sensors],
            "process_time":[s.processing_time for s in self.sensors],
            "waiting":[s.infos.waiting for s in self.sensors],
            "capacity":[s.infos.capacity for s in self.sensors],
            "core":[s.infos.core for s in self.sensors],
            "size":[s.size for s in self.sensors],
            "successed":[s.infos.successed for s in self.sensors],
            "priority":[s.priority for s in self.sensors]
        }
        
        return pd.DataFrame(data)


    def run(self,stop_time = 400):
        self.env.run(until=stop_time)  # launch the simulation for a given duration
        
    
sim = IOT()
sim.run(TIME_STEPS)

df = sim.stats()


#-------------printing--informations--------------------------------------


waited = df.query("t_end_waiting.notna()").query("t_end_waiting.notna()")
cores=df["core"].value_counts()


print("1-------------------------------------------------------------------")
print(f"Total numbers of arrivals is :{df.shape[0]}")
try:
    print(f"rejected percentage {df['successed'].value_counts()['rejected']/df.shape[0]*100:.2f}%")
except:
    print("No rejcted")
print(f'Average waited {(waited["t_end_waiting"]-waited["arrival_time"]).mean():.2f} / Max waited {(waited["t_end_waiting"]-waited["arrival_time"]).max():.2f}')
for core in cores.keys():
    coredf = df.query(f"core=='{core}'").query("t_out.notna()")
    print(f'Percentage of {core} :{coredf["process_time"].sum()/TIME_STEPS*100:.2f}')





4
1-------------------------------------------------------------------
Total numbers of arrivals is :13724
No rejcted
Average waited 0.56 / Max waited 99.88
Percentage of core4 :54.34
Percentage of core2 :54.42
Percentage of core3 :53.72
Percentage of core1 :53.46


In [2]:
intr = df.query("type=='IntrusionDetector'").query("t_end_waiting.notna()")
gas = df.query("type=='GasCarbonDetector'").query("t_end_waiting.notna()")
camera = df.query("type=='Camera'").query("t_end_waiting.notna()")
vocal = df.query("type=='VocalDetector'").query("t_end_waiting.notna()")
temp = df.query("type=='Temperature'").query("t_end_waiting.notna()")
print("\n\n2-----------------------queue statistics--------------------------")
print("")
print("IntrusionDetector  :")
print(f"\twaited mean :{(intr['t_end_waiting']-intr['arrival_time']).mean()}")
print(f"\twaited max :{(intr['t_end_waiting']-intr['arrival_time']).max()}")
print(f"\tProcess mean :",df.query("type =='IntrusionDetector'")["process_time"].mean())
print()
print("GasCarbonDetector  :")
print(f"\twaited mean :{(gas['t_end_waiting']-gas['arrival_time']).mean()}")
print(f"\twaited max :{(gas['t_end_waiting']-gas['arrival_time']).max()}")
print(f"\tProcess mean :",df.query("type =='GasCarbonDetector'")["process_time"].mean())
print()
print("VocalDetector  :")
print(f"\twaited mean :{(vocal['t_end_waiting']-vocal['arrival_time']).mean()}")
print(f"\twaited max :{(vocal['t_end_waiting']-vocal['arrival_time']).max()}")
print(f"\tProcess mean :",df.query("type =='VocalDetector'")["process_time"].mean())
print()
print("Camera  :")
print(f"\twaited mean :{(camera['t_end_waiting']-camera['arrival_time']).mean()}")
print(f"\twaited max :{(camera['t_end_waiting']-camera['arrival_time']).max()}")
print(f"\tProcess mean :",df.query("type =='Camera'")["process_time"].mean())
print()
print("Temperature  :")
print(f"\twaited mean :{(temp['t_end_waiting']-temp['arrival_time']).mean()}")
print(f"\twaited max :{(temp['t_end_waiting']-temp['arrival_time']).max()}")
print(f"\tProcess mean :",df.query("type =='Temperature'")["process_time"].mean())





2-----------------------queue statistics--------------------------

IntrusionDetector  :
	waited mean :0.2315822432273525
	waited max :29.677891311024723
	Process mean : 15.399257929037496

GasCarbonDetector  :
	waited mean :0.07603377428877461
	waited max :11.170368816761766
	Process mean : 5.092715305164826

VocalDetector  :
	waited mean :0.5814843805038118
	waited max :12.36376866903447
	Process mean : 10.066000769320498

Camera  :
	waited mean :0.5498605181385158
	waited max :41.08374712476507
	Process mean : 14.995143187269553

Temperature  :
	waited mean :4.210212705591172
	waited max :99.88398503203643
	Process mean : 5.197375517609762


1-well first thing i see is that Temperature sensor wait way more than the other sensors and that's because it have the lowes priority 
but if we gave him priority than the camera sensor since camera arrive each 10 seconds while Temperature arrive on average each 180 seconds so the Temperature will wait for alot of cameras 
and also Temperature sensor have less process time than the cameras

so i recommend to change priority between camera and Temperature and that will not affect camera too much  but it will reduce the max and average waiting of Temperature sensor alot


on camera.priority = 3 and Temperature.priority = 4
Camera  :
	waited mean :0.4838007330356412
	waited max :29.08446767850546
	Process mean : 15.045186770594144

Temperature  :
	waited mean :4.208874334890464
	waited max :84.94278331912938
	Process mean : 5.057290275839931


on camera.priority = 4 and Temperature.priority = 3
Camera  :
	waited mean :0.5659597453614142
	waited max :29.50515278714738
	Process mean : 14.936890292739934

Temperature  :
	waited mean :1.3155457340287304
	waited max :21.4384930186352
	Process mean : 4.660969350839878

2- (optional) if the user can sacrifice 3 more seconds on average waiting in queue we can use only 3 cores because each core work only around 50% of the time
so if we use 3 cores each core will work around 70% of the time and will have same rejected percentage


Average waited  in 4 cores is 0.46 
Average waited 3 cores is 3.43


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import simpy 
import scipy.stats as st
TIME_STEPS = 24*3600

class Infos():
    def __init__(self,arrival_time,waiting,capacity,test) -> None:
        self.waiting = waiting
        self.capacity = capacity
        self.arrival_time = arrival_time
        self.wait = None
        self.done = None
        self.core = None
        self.test = test
        self.successed = "rejected"


class Sensor():
    id = 0
    def __init__(self,sen_type,priority,size,processing_time,arrival_rate):
        Sensor.id +=1
        self.id = Sensor.id
        self.sen_type = sen_type
        self.size = size
        self.processing_time = processing_time
        self.priority = priority
        self.arrival_rate = arrival_rate
        self.infos :Infos = None



class IntrusionDetector(Sensor):
    def __init__(self):
        size = st.norm(128,32).rvs()
        process_t = st.expon(0,15).rvs()
        arrival_rate = 30
        super().__init__("IntrusionDetector",1,size=size,processing_time=process_t,arrival_rate=arrival_rate)

class GasCarbonDetector(Sensor):
    def __init__(self):
        size = 1
        arrival_rate = 60
        process_t = st.expon(0,5).rvs()
        super().__init__("GasCarbonDetector",1,size=size,processing_time=process_t,arrival_rate=arrival_rate)

class VocalDetector(Sensor):
    def __init__(self):
        size = 256
        process_t = st.expon(0,10).rvs()
        arrival_rate = st.expon(0,3600/12).rvs()
        super().__init__("VocalDetector",2,size=size,processing_time=process_t,arrival_rate=arrival_rate)

class Camera(Sensor):
    def __init__(self):
        size = st.norm(2000,512).rvs()
        process_t = st.expon(0,15).rvs()
        arrival_rate = 10
        super().__init__("Camera",4,size=size,processing_time=process_t,arrival_rate=arrival_rate)

class Temperature(Sensor):
    def __init__(self):
        size = 1
        process_t = st.expon(0,5).rvs()
        arrival_rate = 180
        super().__init__("Temperature",3,size=size,processing_time=process_t,arrival_rate=arrival_rate)



class IOT():
    def __init__(self,cores=["core1","core2","core3","core4"],max_capacity=128000):
        self.cores = cores.copy()
        self.queue = []
        self.queue_capacity = 0
        self.max_capacity = max_capacity
        self.num_cores = len(self.cores)
        self.sensors = []
        self.env = simpy.Environment()
        print(self.num_cores)
        self.res_cores = simpy.Resource(self.env,self.num_cores)
        self.gen = self.env.process(self.sensors_arrival("IntrusionDetector"))
        self.gen = self.env.process(self.sensors_arrival("GasCarbonDetector"))
        self.gen = self.env.process(self.sensors_arrival("VocalDetector"))
        self.gen = self.env.process(self.sensors_arrival("Camera"))
        self.gen = self.env.process(self.sensors_arrival("Temperature"))
    
    def get_capacity(self):
        s = 0
        
        for sen in self.queue:
            s += sen.size
        return s
    
    def get_sensor(self):
        mini = 0
        for idx,sensor in enumerate(self.queue):
            if sensor.priority < self.queue[mini].priority :
                mini = idx
        return mini
    


    def sensors_arrival(self,sensortype):
        while True :

            if sensortype == "IntrusionDetector" :
                sensor=IntrusionDetector()

            elif sensortype == "GasCarbonDetector" :
                sensor=GasCarbonDetector()

            elif sensortype == "Camera" :
                sensor=Camera()
                
            elif sensortype == "VocalDetector" :
                sensor=VocalDetector()
            
            elif sensortype == "Temperature" :
                sensor = Temperature()
            
            

            arrival_rate = sensor.arrival_rate
            yield self.env.timeout(arrival_rate)
            sensor.infos = Infos(self.env.now,len(self.queue),self.get_capacity(),"zit")
            
            self.sensors.append(sensor)
            if sensor.size < (self.max_capacity - self.get_capacity()) :
                sensor.infos.successed = "successed"
                self.queue.append(sensor)
                
            
            self.env.process(self.process_sensors())



                
    
    def process_sensors(self):
          
        with self.res_cores.request() as req :
            
            yield req
            
            if len(self.queue) > 0 :
            
                sensor = self.queue.pop(self.get_sensor())
                sensor.infos.wait = self.env.now
                core = self.cores.pop(0)
                sensor.infos.core = core
                process_rate = sensor.processing_time
                
                yield self.env.timeout(process_rate)

                sensor.infos.done = self.env.now
                
                self.cores.append(core)
            
            
            

    
    def stats(self):
        data ={
            "Id":[s.id for s in self.sensors],
            "type":[s.sen_type for s in self.sensors],
            "arrival_time":[s.infos.arrival_time for s in self.sensors],
            "t_end_waiting":[s.infos.wait for s in self.sensors],
            "t_out":[s.infos.done for s in self.sensors],
            "process_time":[s.processing_time for s in self.sensors],
            "waiting":[s.infos.waiting for s in self.sensors],
            "capacity":[s.infos.capacity for s in self.sensors],
            "core":[s.infos.core for s in self.sensors],
            "size":[s.size for s in self.sensors],
            "successed":[s.infos.successed for s in self.sensors],
            "priority":[s.priority for s in self.sensors]
        }
        
        return pd.DataFrame(data)


    def run(self,stop_time = 400):
        self.env.run(until=stop_time)  # launch the simulation for a given duration
        
    
sim = IOT()
sim.run(TIME_STEPS)

df = sim.stats()


#-------------printing--informations--------------------------------------

intr = df.query("type=='IntrusionDetector'").query("t_end_waiting.notna()")
gas = df.query("type=='GasCarbonDetector'").query("t_end_waiting.notna()")
camera = df.query("type=='Camera'").query("t_end_waiting.notna()")
vocal = df.query("type=='VocalDetector'").query("t_end_waiting.notna()")
temp = df.query("type=='Temperature'").query("t_end_waiting.notna()")
waited = df.query("t_end_waiting.notna()").query("t_end_waiting.notna()")
cores=df["core"].value_counts()


print("1-------------------------------------------------------------------")
print(f"Total numbers of arrivals is :{df.shape[0]}")
try:
    print(f"rejected percentage {df['successed'].value_counts()['rejected']/df.shape[0]*100:.2f}%")
except:
    print("No rejcted")
print(f'Average waited {(waited["t_end_waiting"]-waited["arrival_time"]).mean():.2f} / Max waited {(waited["t_end_waiting"]-waited["arrival_time"]).max():.2f}')
for core in cores.keys():
    coredf = df.query(f"core=='{core}'").query("t_out.notna()")
    print(f'Percentage of {core} :{coredf["process_time"].sum()/TIME_STEPS*100:.2f}')






4
1-------------------------------------------------------------------
Total numbers of arrivals is :13729
No rejcted
Average waited 0.46 / Max waited 27.99
Percentage of core3 :52.80
Percentage of core2 :53.79
Percentage of core4 :53.54
Percentage of core1 :53.36
